# Scotiabank Technical Case

                        Peter Chen

I am provided with more than 1 year of customer product data. The objective is to predict what accounts a customer will acquire in the next month, 2016-05.

The case has the three characters: 1) multi-label 2)time-series 3)Unbalanced and Large dataset.

1) There are 23 labels. One client can have several labels in the same time. It's a multi-label problem, but I decide to train 23 models for each of the labels. If we use multi-label model, when the model trains, the feature importance may affected by different labels, which lead to a bad performance. Also, the unbalanced distribution of labels will affect training. The selected features will be domained by the labels with high frequency.


2) A time-series probelm. Historic accounts information can be used to predict future accounts status. I generate the features of date -- when did the client open the account. The feature could not only used to filter the training and prediction set, but also for prediction. That is to say, if the client open the account before, we should not add them into the training set, because he won't add(it's an action) the account any more. Also, when we choose training set and validation set, we need to focus on the time. Here, I use 2015-03 as training set, use 2015-04 as validation set. When we try to make a prediction of 2015-05.


3) The dataset covers clients and 16 months data, which is more than 3GB. The accounts type data are very unbalanced.I build random forest, and use 'class_weight' to ajust the class.  



In [49]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from sklearn.feature_selection import mutual_info_classif
import numpy as np
from sklearn.metrics import roc_auc_score
import math
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Problem definition

The objective is to predict what accounts a customer will add in the next month. 'Add' is a event.
We define our target Y: {1: The costomer add the account, 0: the customer doesn't add the account}
Note: when we train the model 

# Load Data

The raw data is a large data set. In this notebook, I extract 10,000 sample clients randomly from the client list. Then the analysis can be done in my own computer and run the model locally. After that, I transfer the code on Google Cloud. 

In [2]:
# read test data
df_test = pd.read_csv('test/test_monthly_info.csv')

In [3]:
df_train_accounts = pd.read_csv('sliced_accounts.csv').drop('Unnamed: 0',axis=1)
df_train_info=pd.read_csv('sliced_info.csv').drop(['Unnamed: 0','sex_bin'],axis=1)

In [4]:
#df_train_accounts = pd.read_csv('sliced_accounts.csv').drop('Unnamed: 0',axis=1)

In [5]:
#df_train_info=pd.read_csv('sliced_info.csv').drop('Unnamed: 0',axis=1)

# Data cleaning

## Client Information

In [6]:
# type
df_train_info.dtypes

cust_id                 int64
date                   object
employee_index         object
country                object
sex                    object
age                    object
open_date              object
last_6_months_flag    float64
seniority              object
primary               float64
last_date_primary      object
customer_type          object
customer_relation      object
domestic_index         object
foreigner_index        object
spouse_index           object
channel                object
deceased_status        object
primary_address       float64
province_code         float64
province_name          object
activity_index        float64
gross_income          float64
segment                object
dtype: object

In [7]:
df_train_info.isnull().sum()

cust_id                    0
date                       0
employee_index           367
country                  367
sex                      367
age                        0
open_date                367
last_6_months_flag       367
seniority                  0
primary                  367
last_date_primary     133885
customer_type           1610
customer_relation       1610
domestic_index           367
foreigner_index          367
spouse_index          134059
channel                 1985
deceased_status          367
primary_address          367
province_code           1205
province_name           1205
activity_index           367
gross_income           27005
segment                 2015
dtype: int64

In [8]:
# clean data missing value of age and seniority and transform into numerical data.
# replace missing value by average
df_train_info['age']=df_train_info['age'].replace(' NA','-1')
df_train_info['seniority']=df_train_info['seniority'].replace(' NA','-1')
df_train_info['seniority']=df_train_info['seniority'].replace('     NA','-1')
df_train_info['age']=df_train_info['age'].astype('int')
df_train_info['seniority']=df_train_info['seniority'].astype('int')
df_train_info['age']=df_train_info['age'].replace(-1,df_train_info['age'].mean())
df_train_info['seniority']=df_train_info['seniority'].replace(-1,df_train_info['seniority'].mean())
#fill the missing values of gross_income by average
df_train_info['gross_income']=df_train_info['gross_income'].fillna(df_train_info['gross_income'].mean())

In [9]:
#Before dealing with other missing value, generate a new features which is the difference of last_date_primary and date.
#The new feature day_diff reflects how many days the clients has not been primary.
df_train_info['date']=pd.to_datetime(df_train_info['date'])
df_train_info['open_date']=pd.to_datetime(df_train_info['open_date'])
df_train_info['last_date_primary']=pd.to_datetime(df_train_info['last_date_primary'])
df_train_info['day_diff_primary']=(df_train_info['date']-df_train_info['last_date_primary']).fillna(0).map(lambda x: int(x.days/30))
df_train_info['day_diff_open']=(df_train_info['date']-df_train_info['open_date']).fillna(0).map(lambda x: int(x.days/30))
df_train_info['date']=df_train_info['date'].astype('str')

In [ ]:
# age, seniority and gross_income are three useful numerical data. We need to do segmentation for better features.
# After one-hot encoding, they will become binary features.
def age_bin(age): # age is classifed by life cycle
    if age<18:
        return 0
    elif age>=18 and age <23:
        return 1
    elif age>=23 and age < 30:
        return 2
    elif age>=30 and age <40:
        return 4
    elif age>=40 and age <=60:
        return 5
    elif age>60:
        return 6
def seniority_bin(seniority): # seniority is classifed by customer life cycle
    if seniority ==0:
        return 0
    elif seniority <3 and seniority >0:
        return 1
    elif seniority> 3 and seniority<=6:
        return 2
    elif seniority ==6:
        return 3
    elif seniority>6 and seniority<12:
        return 4
    elif seniority ==12:
        return 5
    elif seniority >12:
        return 6
def income_bin(gross_income): # income is classifed by percentiles
    if gross_income < 76403:
        return 0
    elif gross_income >= 76403 and gross_income<123231:
        return 1
    elif gross_income >= 123231 and gross_income<136899:
        return 2
    elif gross_income > 136899:
        return 3

In [ ]:
# day_diff_open feature shows whether clients who open the account in one months.
df_train_info['day_diff_open']=X_block_14['day_diff_open'].apply(lambda x: 1 if x<31 else 0 )
# Apply bin functions
df_train_info['age_bin']=X_block_14['age'].apply(age_bin)
df_train_info['seniority_bin']= X_block_14['seniority'].apply(seniority_bin)
df_train_info['income_bin']= X_block_14['gross_income'].apply(income_bin)
df_train_info=X_block_14.drop(['age','seniority','province_code'],axis=1)

In [10]:
# fill all missing values with categorial features with -1. Then do one-hot encoding. 
df_train_info=df_train_info.fillna(-1)

In [ ]:
#One-hot encoding for features of client information.
df_train_info2=pd.get_dummies(df_train_info, columns = ['employee_index', 'country', 'sex',
        'last_6_months_flag', 'primary',
       'customer_type', 'customer_relation',
       'domestic_index', 'foreigner_index', 'spouse_index', 'channel',
       'deceased_status', 'primary_address','province_name',
       'activity_index', 'segment','age_bin', 'seniority_bin'])

In [11]:
df_train_info.head(2)

,cust_id,date,employee_index,country,sex,age,open_date,last_6_months_flag,seniority,primary,...,channel,deceased_status,primary_address,province_code,province_name,activity_index,gross_income,segment,day_diff_primary,day_diff_open
0,1367111,2015-07-28,N,ES,V,25.0,2014-12-09 00:00:00,0.0,7.0,1.0,...,KHE,N,1.0,28.0,MADRID,0.0,48574.8,03 - UNIVERSITARIO,0,7
1,1367111,2015-08-28,N,ES,V,25.0,2014-12-09 00:00:00,0.0,8.0,1.0,...,KHE,N,1.0,28.0,MADRID,0.0,48574.8,03 - UNIVERSITARIO,0,8


In [12]:
df_train_info.describe()

,cust_id,age,last_6_months_flag,seniority,primary,primary_address,province_code,activity_index,gross_income,day_diff_primary,day_diff_open
count,1.340850e+05,134085.000000,134085.000000,134085.000000,134085.000000,134085.000000,134085.000000,134085.000000,1.340850e+05,134085.000000,134085.000000
mean,8.264522e+05,40.195903,0.058112,80.078843,1.140702,0.994526,26.291353,0.462878,1.340974e+05,-0.000582,81.670425
std,4.304682e+05,17.091791,0.245377,66.211474,3.783715,0.104491,13.084682,0.504081,1.778989e+05,0.047455,68.020026
min,1.589800e+04,2.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,7.445640e+03,-6.000000,0.000000
25%,4.457010e+05,24.000000,0.000000,23.000000,1.000000,1.000000,15.000000,0.000000,7.640391e+04,0.000000,23.000000
50%,9.251050e+05,39.000000,0.000000,51.000000,1.000000,1.000000,28.000000,0.000000,1.232311e+05,0.000000,52.000000
75%,1.192617e+06,51.000000,0.000000,136.000000,1.000000,1.000000,34.000000,1.000000,1.368992e+05,0.000000,140.000000
max,1.547874e+06,108.000000,1.000000,255.000000,99.000000,1.000000,52.000000,1.000000,8.658967e+06,0.000000,259.000000


## Target Definition and Feature of Accounts

As we mentioned before.
We define our target Y: {1: The costomer add the account, 0: the customer doesn't add the account}
Therefore, we need to get the earliest opening date, which could help converting the accounts data to our target.

In accounts table, we can get th historic accounts status of each clients. We want to extract the earlist date of each type of accounts the customer open. For example, If a clients added a saving account last month, the feature [saving_account_first] will be 1. if a clients added a tax account 2 months ago, [tax_account_first] will be 2 ,etc.

Target Y will be 1 if [account_first] is 0 and he opens the account this month. Target Y will be 0 for others which include a) clients who never opened the account and don't open account this month  b)clients who open the account before. Before training, we will filter b) clients by excluding the clients whose [account_first]>0

In [13]:
df_train_accounts['date']=pd.to_datetime(df_train_accounts['date'])

In [14]:
df_train_accounts2=df_train_accounts.copy()

In [16]:
df_train_accounts['date']=pd.to_datetime(df_train_accounts['date'])
for i in tqdm(list(df_train_accounts.columns)[3:]):
    # look for the min date of opening the account. if a client never opened the account before, the feature will be 0.
    df_date= df_train_accounts[df_train_accounts[i]==1].groupby('cust_id').date.min().reset_index().rename(index=str, columns={"date": i+'_first'})
    df_train_accounts = df_train_accounts.merge(df_date,how='left',on='cust_id') 
    label_name= i+'_target'
    new_i=i+'_first'
    df_train_accounts[new_i]=pd.to_datetime(df_train_accounts [new_i])
    df_train_accounts[new_i]=df_train_accounts['date'] - df_train_accounts[new_i]
    df_train_accounts[new_i]=df_train_accounts[new_i].fillna(0)
    df_train_accounts[new_i]=df_train_accounts[new_i].apply(lambda x: math.ceil(x.days/31))
    df_train_accounts[new_i]=df_train_accounts[new_i].apply(lambda x: 0 if x <0 else x)
    df_train_accounts[label_name]=df_train_accounts.apply(lambda x: 1 if (x[i] == 1 and x[new_i]==0) else 0,axis=1)
df_train_accounts['date']=df_train_accounts['date'].astype('str')

100%|██████████| 23/23 [01:32<00:00,  4.24s/it]


In [17]:
df_train_accounts['date']=df_train_accounts['date'].astype('str')

In [18]:
df_train_accounts.columns

Index(['cust_id', 'cust_key', 'date', 'savings_account', 'guarantees',
       'current_accounts', 'derived_account', 'payroll_account',
       'junior_account', 'more_particular_account', 'particular_account',
       'particular_plus_account', 'short_term_deposits',
       'medium_term_deposits', 'long_term_deposits', 'e_account', 'funds',
       'mortgage', 'pensions', 'loans', 'taxes', 'credit_card', 'securities',
       'home_account', 'payroll', 'direct_debt', 'savings_account_first',
       'savings_account_target', 'guarantees_first', 'guarantees_target',
       'current_accounts_first', 'current_accounts_target',
       'derived_account_first', 'derived_account_target',
       'payroll_account_first', 'payroll_account_target',
       'junior_account_first', 'junior_account_target',
       'more_particular_account_first', 'more_particular_account_target',
       'particular_account_first', 'particular_account_target',
       'particular_plus_account_first', 'particular_plus_accou

In [27]:
# fill missing values of payroll accounts by 0
df_train_accounts=df_train_accounts.fillna(0)

In [28]:
# convert month into month block.(eg. 2015-07-28 is 1, '2015-08-28' is 2, etc. )
dates=list(df_train_accounts['date'].unique())
df_train_accounts['month_block']=df_train_accounts['date'].apply(lambda x: dates.index(x))

In [41]:
df_merge=df_train_accounts.merge(df_train_info2,how='left',on=['cust_id','date'])
df_merge=df_merge.drop(['open_date','last_date_primary','province_code'],axis=1)

## Feature selection

After feature engineering, we generate 410 features. A good feature selection is very helpful to reduce noise and get a high accuracy or recall. The following code is Mutual information used to do feature selection. We get top 50 features that has highest  Mutual information values for each label. By doing this loop for every label, we can get a set of good features, which will be applied into the model. 

In [270]:
def MI(topk,df,features, labels): 
    topk_features=[]
    for label in tqdm(labels):
        print(label)
        scores=[]
        for feature in features:
            score=mutual_info_classif(df[[feature]],df[label])
            scores.append(score[0])
          #  print(feature, ':',score)
        df_mi=pd.DataFrame({'features':features,'MI':scores})
        topk_feature=df_mi.sort_values('MI',ascending= False)['features'][:topk]
        topk_features.extend(list(topk_feature))
    return list(set(topk_features))

In [271]:
features=MI(50,train,X_columns,targets_columns)

  0%|          | 0/23 [00:00<?, ?it/s]

savings_account_target


  4%|▍         | 1/23 [07:57<2:55:04, 477.48s/it]

guarantees_target


  9%|▊         | 2/23 [14:11<2:36:12, 446.31s/it]

...

 91%|█████████▏| 21/23 [1:33:36<08:00, 240.21s/it]

payroll_target


 96%|█████████▌| 22/23 [1:37:24<03:56, 236.59s/it]

direct_debt_target


100%|██████████| 23/23 [1:41:19<00:00, 236.12s/it]


In [272]:
features

['deceased_status_-1',
 'province_name_BIZKAIA',
 'primary_address_1.0',
 'channel_KHM',
 'province_name_CACERES',
 'channel_KAW',
 'customer_relation_-1',
 'province_name_CORDOBA',
 'province_name_NAVARRA',
 'province_name_PALMAS, LAS',
 'province_name_CADIZ',
 'channel_KCH',
 'country_PA',
 'province_name_CEUTA',
 'last_6_months_flag_0.0',
 'channel_KAT',
 'channel_KFP',
 'channel_KCA',
 'province_name_TARRAGONA',
 'channel_KHC',
 'channel_KAG',
 'activity_index_1.0',
 'province_name_TOLEDO',
 'junior_account_first',
 'segment_02 - PARTICULARES',
 'country_DE',
 'foreigner_index_-1',
 'derived_account_first',
 'channel_KBS',
 'customer_type_2',
 'channel_KHK',
 'sex_-1',
 'channel_-1',
 'country_MA',
 'domestic_index_-1',
 'segment_03 - UNIVERSITARIO',
 'province_name_ALICANTE',
 'province_name_ZAMORA',
 'activity_index_0.0',
 'credit_card_first',
 'country_CN',
 'province_name_SEVILLA',
 'particular_account_first',
 'province_name_GIRONA',
 'channel_KAE',
 'province_name_RIOJA, LA',

# Modelling & Evaluation

Here I decided to use the random forest. The modell and feature engineering are operated on google cloud because the large dataset lead to a memory error if running locally.
The reason why I choose random forest:


1) Good performance. As a ensembling model, it reduced the variance to reduce overfitting. 


2) It also has a function of feature selection. we can evaluate the feature importance if we are asked to show the interpretability by stakeholders. 


3) It has parameter of clasee_weight, which we can use to balance our dataset. Also it support multi-label modelling, though we don't use it here.

In the model, we mainly focus on a high recall, which equip us the ability to capture the clients behavior. Because the model is a recommendation system, instead of anti-fraud model, we don't have to ask for a low false positive(FP). But we still hope FP won't too high because of disturbing and cost. It's a trade-off. 

The model run on google cloud. The recall and FP result didn't showed here. But you can get the performance by evaluating my submission file. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
for i in range(0,3): # Because memory restriction, we seperate the labels by three. 
    labels_names_sub=labels_names[8*i:8*(i+1)]
    print("the ith 8 targets",i)
    for label_name in tqdm(labels_names_sub): # pick one label each time and train the model.
        label_name_first=label_name+'_first'
        label_name_target=label_name+'_target'
        
        # filter the training set and validation set
        Y_block_14_ca=Y_block_14[X_block_14[label_name_first]==0]
        Y_block_15_ca=Y_block_15[X_block_15[label_name_first]==0]
        X_block_14_ca=X_block_14[X_block_14[label_name_first]==0]
        X_block_15_ca=X_block_15[X_block_15[label_name_first]==0]
        
        #feature selection by feature importance
        clf = RandomForestClassifier(n_estimators=600, max_depth=3,class_weight= 'balanced',random_state=88,n_jobs=-1)
        clf.fit(X_block_14_ca,Y_block_14_ca[label_name_target])
        feature_importance=pd.DataFrame({'name':list(X_block_14_ca.columns),'importance':clf.feature_importances_}).sort_values('importance',ascending=False)
        # take best 100 features into the model
        good_features=feature_importance['name'][0:100]
        
        # Train the model by 100 selected features 
        clf = RandomForestClassifier(n_estimators=600, max_depth=9,class_weight= 'balanced',random_state=88,n_jobs=-1)
        clf.fit(X_block_14_ca[good_features],Y_block_14_ca[label_name_target])
        
        # predict the label
        y_pred = clf.predict(X_block_15_ca[good_features])
        
        #calculate the recall of the label.
        recall=recall_score(Y_block_15_ca[label_name_target],y_pred)
        print(label_name_target,Y_block_15_ca[label_name_target].sum(),y_pred.sum(),recall)
        y_pred_prob = clf.predict_proba(X_block_16[good_features])
        Y_block_16[label_name_target]=y_pred_prob[:,-1]
    labels_names_sub.append('cust_id')
    #save the result
    Y_block_16.iloc[:,8*i:8*(i+1)].to_csv('sub_result_final'+str(i)+'.csv',index=True, header=True)
    print('successful save file'+'sub_result_final'+str(i)+'.csv')

In [ ]:
from sklearn.metrics import recall_score
def recall(y_test,y_pred):
    recalls=[]
    for i in range(y_test.shape[1]):
        recall=recall_score(y_test.iloc[:,i],y_pred[:,i])
        recalls.append(recall)
        #print(recall)
        average=np.mean(recalls)
    return average

# Organize result

In [ ]:
def get_business(x):
    sorted_buss=x[1:-1].sort_values(ascending=False)
    business = sorted_buss[sorted_buss>0.5].index.tolist()
    if len(business)==0:
        business.append(sorted_buss.index[0])
    return '; '.join(business)

In [ ]:
result_int['products']=result_int.apply(get_business,axis=1)

In [22]:
result_int[['cust_id','products']].to_csv('{Peter_Chen}_predictions2.csv',index=False, header=True)

# Questions & Answer

## 1.	Provide a brief summary at the end to explain the additional steps you could have taken to increase your model(s) ability to make correct predictions.



The model is applicable in the business of banks. It doesn't require a high GPU to maintain and save the conputation resouce. Also, in terms of interpretability, the model could give an insight to show key factors to affect a product. In addition, each label has it's own model, different label of business line won't affect each other but shared the same features, which allowed us to maintain and update each product model easily. The logic method applied in the case is to transer a time-series problem into classification problem. It's the key to define the model. 

Measures taken to improve the model:

1) Hyper parameter tuning. Because of large dateset and limited computation resource, I didn't do a lot of hyper papameter tuning. Train the model with more samples will reduce overfitting but require more computation resource. A distributed system can easily solve the problem and allow us to do hyper parameter tuning.

2)Generate more features. Explore more on the bin-feature and select the features carefully for each labels and models will definetly be helpful. 

3) Model Stacking. It's a very common method applied in kaggle competition. We can also build a user-user recommendation system to predict the product. Average the predicting probability will increase the score.



## 2.	Which part of the business process could you apply this predictive model to?

Recommendation system

This model is able to predict the demand of the clients next month based on historic data and demographic data. For high True positive labels, we can ask clients representative to have a cold call, which is very helpful in improving the conversion rate of clients. For low true positive labels, we can send email or push app notification, which is able to save money in marketing.

Business process:

1. Cold Start. When a new client opens the account in our bank, we can introduce him the predicted products.

2. Marketing. When we have a new promotion on some specific product, we can send the notification to the labeled clients and avoiding disturbing the clients without interest.

3. Cold call. The model can make cold call of sales more efficient and also enable telephone customer service to know better the clients.
